In [0]:
import pandas as pd
import numpy as np

In [0]:
test = pd.read_csv('profile_data_test.csv')
train = pd.read_csv('profile_data_train.csv')

all_data = pd.concat((test.loc[:,'handle':'avg_word_count'], train.loc[:,'handle':'avg_word_count'])).reset_index(drop=True)

In [0]:
X_train = all_data[:train.shape[0]]
X_test = all_data[train.shape[0]:]

y_train = train['age_group']
y_test = test['age_group']

In [7]:
# first try xgboost on numerical features only
import copy

X_train_num = copy.deepcopy(X_train)
del X_train_num['handle']
del X_train_num['tweets_text']
del X_train_num['hashtags']

X_train_num.head()

,followers,following,follower_following_ratio,avg_favorites,min_favorites,max_favorites,avg_retweets,min_retweets,max_retweets,hashtags_per_tweet,mentions_per_tweet,avg_timedelta_hrs,avg_word_count
0,126790.0,38762.0,3.270987,367.000,207.0,663.0,31.800,18.0,64.0,0.50,0.1,26.393673,166
1,1272.0,426.0,2.985915,10.700,0.0,55.0,6.300,0.0,52.0,0.20,0.2,217.188333,81
2,74947.0,49.0,1529.530612,181.100,42.0,774.0,15.400,3.0,53.0,0.50,0.1,88.825556,386
3,560650.0,119.0,4711.344538,23.400,4.0,94.0,1.400,0.0,6.0,0.10,0.0,1.508302,104
4,392302.0,3048.0,128.708005,210.125,3.0,514.0,47.125,2.0,141.0,0.75,0.5,20.227262,281


In [8]:
X_test_num = copy.deepcopy(X_test)
del X_test_num['handle']
del X_test_num['tweets_text']
del X_test_num['hashtags']

X_test_num.head()

,followers,following,follower_following_ratio,avg_favorites,min_favorites,max_favorites,avg_retweets,min_retweets,max_retweets,hashtags_per_tweet,mentions_per_tweet,avg_timedelta_hrs,avg_word_count
1699,3946343.0,69.0,57193.376812,5167.4,131.0,24727.0,163.5,1.0,729.0,0.7,0.1,8.068981,109
1700,54359.0,17.0,3197.588235,548.5,234.0,1230.0,58.7,17.0,110.0,0.0,0.0,8.208519,54
1701,778.0,659.0,1.180577,2.6,0.0,13.0,0.3,0.0,2.0,0.0,0.0,20.997809,121
1702,114.0,148.0,0.770270,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7.273642,54
1703,256.0,207.0,1.236715,4.4,0.0,28.0,1.2,0.0,5.0,0.0,0.1,490.341358,76


# Try XGBoost

In [0]:
from xgboost import XGBClassifier

xgb = XGBClassifier()

In [10]:
xgb.fit(X_train_num, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [11]:
from sklearn.metrics import accuracy_score

accuracy_score(y_test, xgb.predict(X_test_num))

0.2823529411764706

In [0]:
from sklearn.model_selection import GridSearchCV

In [13]:
xgb.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

In [0]:
params = {'learning_rate': np.arange(0.1, 0.4, 0.05),
          'max_depth': np.arange(3, 9, 1),
          'n_estimators': np.arange(500, 600, 100)}

In [0]:
clf = GridSearchCV(xgb, params, scoring='accuracy', n_jobs=-1, verbose=2, cv=3)

In [23]:
clf.fit(X_train_num, y_train)

Fitting 3 folds for each of 42 candidates, totalling 126 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  37 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 126 out of 126 | elapsed:  5.5min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                     colsample_bylevel=1, colsample_bynode=1,
                                     colsample_bytree=1, gamma=0,
                                     learning_rate=0.1, max_delta_step=0,
                                     max_depth=3, min_child_weight=1,
                                     missing=None, n_estimators=100, n_jobs=1,
                                     nthread=None, objective='multi:softprob',
                                     random_state=0, reg_alpha=0, reg_lambda=1,
                                     scale_pos_weight=1, seed=None, silent=None,
                                     subsample=1, verbosity=1),
             iid='deprecated', n_jobs=-1,
             param_grid={'learning_rate': array([0.1 , 0.15, 0.2 , 0.25, 0.3 , 0.35, 0.4 ]),
                         'max_depth': array([3, 4, 5, 6, 7, 8]),
                      

In [24]:
clf.best_estimator_.get_params

<bound method XGBModel.get_params of XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.30000000000000004, max_delta_step=0, max_depth=6,
              min_child_weight=1, missing=None, n_estimators=500, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)>

In [39]:
accuracy_score(y_test, clf.best_estimator_.predict(X_test_num))

0.31058823529411766

In [0]:
clf.best_estimator_.n_estimators = 100

In [53]:
accuracy_score(y_test, clf.best_estimator_.predict(X_test_num))  # n estimators 100 same as n estimators 500

0.31058823529411766

# Try Catboost

In [55]:
!pip install catboost

     |████████████████████████████████| 64.8MB 59kB/s 


In [0]:
from catboost import CatBoostClassifier

In [0]:
cb = CatBoostClassifier()

In [0]:
cb.fit(X_train_num, y_train)

In [71]:
accuracy_score(y_test, cb.predict(X_test_num))

0.3058823529411765